# Export Image Classification model

https://github.com/aws/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/imageclassification_caltech/Image-classification-transfer-learning-highlevel.ipynb

In [ ]:
model_data = 's3://...'
prefix = 'export-imageclassification/'

In [ ]:
%%sh -s $model_data $prefix
aws s3 cp $1 $2
cd export-imageclassification
tar xvfz model.tar.gz

In [ ]:
import json
import mxnet as mx
from mxnet import gluon

sym_json = json.load(open(prefix+'image-classification-symbol.json'))
sym_json_string = json.dumps(sym_json)

In [ ]:
net = gluon.nn.SymbolBlock(
    outputs=mx.sym.load_json(sym_json_string),
    inputs=mx.sym.var('data'))

In [ ]:
%%sh
apt-get update
apt-get -y install graphviz
pip install -q graphviz

In [ ]:
mx.viz.plot_network(
    net(mx.sym.var('data'))[0],   
    node_attrs={"shape":"oval","fixedsize":"false"})

In [ ]:
net.load_parameters(prefix+'image-classification-0002.params', allow_missing=True)
net.collect_params().initialize()

In [ ]:
test_sample = mx.ndarray.random.normal(shape=(1,3,224,224))    # Random image

response = net(test_sample)
print(response.shape)    # A vector of 257 probabilities